In [80]:
import pandas as pd
import folium
from IPython.display import display, IFrame

In [4]:
US_states_json= "../data/processed/us-states.json"

In [37]:
# 1. Import necessary libraries
import pandas as pd
import folium
import json


In [45]:
# 2. Load the GeoJSON and CSV datasets
#    - us-states.json contains state boundaries
#    - correct_dtypes_merged_fcc_census_1940.csv has 1939 station counts + 1940 census
#    - correct_dtypes_merged_fcc_census_1950.csv has 1950 station counts + 1950 census
with open('../data/processed/us-states.json') as f:
    states_geo = json.load(f)  # :contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}

df1940 = pd.read_csv('../data/processed/correct_dtypes_merged_fcc_census_1940.csv')
df1950 = pd.read_csv('../data/processed/correct_dtypes_merged_fcc_census_1950.csv')


In [47]:
print(df1940.columns)
print(df1950.columns)

Index(['state', 'total', 'Population_1940', 'radio_per_100k'], dtype='object')
Index(['State', 'AM', 'FM_Commercial', 'FM_Educational', 'TV', 'Total',
       'Population_1950', 'total_radio', 'radio_per_100k'],
      dtype='object')


In [50]:
# Suppose after inspection your key columns are:
#   'state'              – state name matching the GeoJSON "properties.name"
#   'total_stations'     – number of radio stations in that year
#   'population'         – census population for that year
#   'stations_per_100k'  – computed stations per 100,000 people
#
# If your columns differ, rename them like this:
df1940 = df1940.rename(columns={
    'total': 'total_stations',
    'Population_1940': 'population',
    'radio_per_100k': 'stations_per_100k'
})
df1950 = df1950.rename(columns={
    'State': 'state',
    'total_radio': 'total_stations',
    'Population_1950': 'population',
    'radio_per_100k': 'stations_per_100k'
})



In [52]:
print(df1940.columns)
print(df1950.columns)

Index(['state', 'total_stations', 'population', 'stations_per_100k'], dtype='object')
Index(['state', 'AM', 'FM_Commercial', 'FM_Educational', 'TV', 'Total',
       'population', 'total_stations', 'stations_per_100k'],
      dtype='object')


In [106]:
# 4. Define a function to create and save a choropleth map
def make_choropleth(df, value_column, legend, outfile):
    """
    df            : DataFrame with 'state' + value_column
    value_column  : name of column to visualize (e.g. 'total_stations' or 'stations_per_100k')
    legend        : legend title (string)
    outfile       : filename to save the HTML map to
    """
    # Center USA map
    m = folium.Map(location=[37.8, -96], zoom_start=4)
    folium.Choropleth(
    geo_data=states_geo,
    name='choropleth',
    data=df,
    columns=['state', value_column],
    key_on='feature.properties.name',
    fill_color='Spectral', ##change depending on map a, b, c, or d
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name=legend
    ).add_to(m)

    folium.LayerControl().add_to(m)
    m.save(outfile)
    print(f"Saved map to {outfile}")
    
    display(IFrame(src=outfile, width='100%', height=500))



In [ ]:
# 5. Generate the four maps
#    Map A: Total stations in 1939 (using 1940 census file)

In [108]:
make_choropleth(
    df1940, 
    value_column='total_stations', 
    legend='Total Radio Stations (1939)', 
    outfile='../output/stations_1939_total.html'
)

#map color was spectral: because its a diverging spectrum and there's a lot of middling values, and extremes: keep it spectral



Saved map to ../output/stations_1939_total.html


In [96]:
#    Map B: Stations per 100k in 1939
make_choropleth(
    df1940, 
    value_column='stations_per_100k', 
    legend='Radio Stations per 100k (1939)', 
    outfile='../output/stations_1939_per100k.html'
)

#changed color palette to YlGnBu in the 

Saved map to ../output/stations_1939_per100k.html


In [72]:
#    Map C: Total stations in 1950
make_choropleth(
    df1950, 
    value_column='total_stations', 
    legend='Total Radio Stations (1950)', 
    outfile='stations_1950_total.html'
)


Saved map to stations_1950_total.html


In [70]:
#    Map D: Stations per 100k in 1950
make_choropleth(
    df1950, 
    value_column='stations_per_100k', 
    legend='Radio Stations per 100k (1950)', 
    outfile='stations_1950_per100k.html'
)

Saved map to stations_1950_per100k.html
